# Drift Diffusion Modeling Workshop - Colab Notebook (Using PyDDM)


## Setup
First, we install the `pyddm` package if it's not already available in this Colab environment.
Then we get the simulated data file from github.

In [ ]:
!pip install pyddm --quiet
!wget https://raw.githubusercontent.com/mehdisenoussi/misc_utils/refs/heads/main/sim_dataset/ddm_sample_fitting-cont_data.csv -O my_data.csv


--2025-06-08 12:18:25--  https://raw.githubusercontent.com/mehdisenoussi/misc_utils/refs/heads/main/sim_dataset/ddm_sample_fitting-cont_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103174 (101K) [text/plain]
Saving to: ‘my_data.csv’

my_data.csv         100%[===================>] 100.76K  --.-KB/s    in 0.01s   

2025-06-08 12:18:25 (10.3 MB/s) - ‘my_data.csv’ saved [103174/103174]




## Imports
Here are the essential packages we will use.

In [ ]:
import numpy as np
import pyddm
import pyddm.plot as ddmplot
import matplotlib.pyplot as plt
from pyddm import Model, Sample
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter
from pyddm.functions import fit_adjust_model


## Loading the data file and making it into a pyddm Sample

In [ ]:
import pandas as pd
from pyddm import Sample

df = pd.read_csv("my_data.csv")

# Create a Sample from response time and correct/error columns in the data file
sample_real = Sample.from_pandas_dataframe(df, rt_column_name="rt", choice_column_name="response")



## Create the model (with some correct parameters from the original simulation) to try to identify the true drift and bound parameters!

In [ ]:

from pyddm import Fittable, Fitted
from pyddm.models import Drift, Bound, Overlay, Noise, OverlayChain, OverlayUniformMixture

class DriftFit(Drift):
    name = "Fittable drift"
    required_parameters = ["drift"]
    required_conditions = []
    def get_drift(self, x, t, conditions, **kwargs):
        return self.drift

class BoundFit(Bound):
    name = "Fittable bound"
    required_parameters = ["bound"]
    required_conditions = []
    def get_bound(self, x, t, conditions, **kwargs):
        return self.bound


ddm_model = Model(
    drift=DriftFit(drift=Fittable(minval=0, maxval=2)),
    noise=NoiseConstant(noise=.5),
    bound=BoundFit(bound=Fittable(minval=0, maxval=2)),
    overlay=OverlayNonDecision(nondectime=.3),
    IC=ICPointSourceCenter(), T_dur=5
)



## Interactive interface

In [ ]:
import pyddm.plot as ddmplot
ddmplot.model_gui_jupyter(ddm_model, sample=sample_real)

Output()